In [1]:
from lxml import etree, html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil

# Mueve todos los fichero del directorio de trabajo a otro de archivo

source_dir = './BOEs'
target_dir = './BOEs_Anteriores'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    shutil.move(os.path.join(source_dir, file_name), target_dir)

# Recoge resumen diario del BORME de hoy

In [2]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210204
dia = 04
mes = 02
año = 2021
04/02/2021


In [3]:
URL_XML_resumen =  "https://www.boe.es/diario_borme/xml.php?id=BORME-S-" + str(hoy)

In [4]:
URL_XML_resumen

'https://www.boe.es/diario_borme/xml.php?id=BORME-S-20210204'

In [5]:
url = URL_XML_resumen
r = requests.get(url)

In [7]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

save_html(r.content, './BORMEs/Resumen-BORME-' + hoy + '.xml')

In [9]:
resumen = etree.parse('./BORMEs/Resumen-BORME-' + hoy + '.xml')

In [10]:
raiz=resumen.getroot()
raiz.tag
raiz_sumario = raiz

In [11]:
seccion = raiz.findall("sumario/diario/seccion")
seccion

[]

In [12]:
for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')
    print(nombre_seccion)

['SECCIÓN PRIMERA. Empresarios']
['SECCIÓN PRIMERA. Empresarios']
['SECCIÓN SEGUNDA. Anuncios y avisos legales']


In [32]:
for seccion in raiz.xpath('//seccion'):
    nombre_num = seccion.xpath('@num')
    nombre_seccion = seccion.xpath('@nombre')
    print(str(nombre_num) + str(nombre_seccion))
    for emisor in seccion:
        nombre_emisor = emisor.xpath('@nombre')
        print('\t' + str(nombre_emisor))

['A']['SECCIÓN PRIMERA. Empresarios']
	['Actos inscritos']
['B']['SECCIÓN PRIMERA. Empresarios']
	['Otros actos publicados en el Registro Mercantil']
['C']['SECCIÓN SEGUNDA. Anuncios y avisos legales']
	['CONVOCATORIAS DE JUNTAS']
	['FUSIONES Y ABSORCIONES DE EMPRESAS']
	['ESCISIÓN DE EMPRESAS']
	['AUMENTO DE CAPITAL']
	['REDUCCIÓN DE CAPITAL']
	['OTROS ANUNCIOS Y AVISOS LEGALES']


In [34]:
tabla_resumen = pd.DataFrame()

In [36]:
for seccion in raiz.xpath('//seccion[contains(@nombre, "SECCIÓN SEGUNDA. Anuncios y avisos legales")]'):
    nombre_seccion = seccion.xpath('@nombre')

    for emisor in seccion:
        nombre_emisor = emisor.xpath('@nombre') 

        for item in emisor:
            item_id = item.xpath('@id')
            item_name = item.xpath('.//titulo/text()') 
            item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]
            tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                  'Emisor' : nombre_emisor,
                                                  'Item_id': item_id, 
                                                  'Item_Nombre' : item_name, 
                                                  'Item_URL_XML' : item_urlXml},
                                                  ignore_index=True)


In [37]:
tabla_resumen.sort_values('Item_id')

,Emisor,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,[CONVOCATORIAS DE JUNTAS],"[COLEGIO NUESTRA SEÑORA SANTA MARÍA, S.L.]",https://www.boe.es/diario_borme/xml.php?id=BOR...,[BORME-C-2021-408],[SECCIÓN SEGUNDA. Anuncios y avisos legales]
1,[CONVOCATORIAS DE JUNTAS],"[PRODUCCIONS LLEVANT, S.A.\r\n(EN LIQUIDACIÓN)]",https://www.boe.es/diario_borme/xml.php?id=BOR...,[BORME-C-2021-409],[SECCIÓN SEGUNDA. Anuncios y avisos legales]
2,[CONVOCATORIAS DE JUNTAS],"[RAÍZ CUADRADA, S.L.]",https://www.boe.es/diario_borme/xml.php?id=BOR...,[BORME-C-2021-410],[SECCIÓN SEGUNDA. Anuncios y avisos legales]
3,[FUSIONES Y ABSORCIONES DE EMPRESAS],"[EQUIPAMIENTOS TÉCNICOS COMERCIALES, S.A. \r\n...",https://www.boe.es/diario_borme/xml.php?id=BOR...,[BORME-C-2021-411],[SECCIÓN SEGUNDA. Anuncios y avisos legales]
4,[FUSIONES Y ABSORCIONES DE EMPRESAS],"[HORMIGONES ARGA, SOCIEDAD ANÓNIMA \r\n(SOCIED...",https://www.boe.es/diario_borme/xml.php?id=BOR...,[BORME-C-2021-412],[SECCIÓN SEGUNDA. Anuncios y avisos legales]
5,[ESCISIÓN DE EMPRESAS],"[MENDESKO &amp; ASOCIADOS, S.L.\r\n(SOCIEDAD E...",https://www.boe.es/diario_borme/xml.php?id=BOR...,[BORME-C-2021-413],[SECCIÓN SEGUNDA. Anuncios y avisos legales]
6,[AUMENTO DE CAPITAL],"[WOLROC 2015, S.L.]",https://www.boe.es/diario_borme/xml.php?id=BOR...,[BORME-C-2021-414],[SECCIÓN SEGUNDA. Anuncios y avisos legales]
7,[REDUCCIÓN DE CAPITAL],"[COMPANYIA DEL FERROCARRIL D'OLOT A GIRONA,\r\...",https://www.boe.es/diario_borme/xml.php?id=BOR...,[BORME-C-2021-415],[SECCIÓN SEGUNDA. Anuncios y avisos legales]
8,[REDUCCIÓN DE CAPITAL],"[MARIE CLAIRE, S.A.]",https://www.boe.es/diario_borme/xml.php?id=BOR...,[BORME-C-2021-416],[SECCIÓN SEGUNDA. Anuncios y avisos legales]
9,[REDUCCIÓN DE CAPITAL],"[MATADERO DE CUMBRES MAYORES, SOCIEDAD ANÓNIMA]",https://www.boe.es/diario_borme/xml.php?id=BOR...,[BORME-C-2021-417],[SECCIÓN SEGUNDA. Anuncios y avisos legales]


# Descarga ficheros XML asociados

In [ ]:
for item_URL in tabla_resumen['Item_URL_XML']:
    r = requests.get(item_URL)
    #f = './BOEs/' + item_URL[-16:] + '.xml'
    ### Separa el número del BOE del resto de la cadena y aplica expresión REGEX 
    print(item_URL.split('='))
    print(re.match('BOE\-A\-[0-9]+\-[0-9]+',item_URL.split('=')[1]))
    filename = re.match('BOE\-A\-[0-9]+\-[0-9]+',item_URL.split('=')[1]).group()
    f = './BOEs/' + filename + '.xml'
    save_html(r.content, f)

# Importa de nuevo los XML generados

In [ ]:

import glob
BOEs = glob.glob('./BOEs/BOE*.xml')
print (BOEs)

# Genera DF con datos Análisis de cada XML

In [ ]:
tabla_analisis = pd.DataFrame()

for BOE in BOEs:
    #print (BOE)
    BOE_XML = etree.parse(BOE)
    raiz=BOE_XML.getroot()

    materias = [materia.text for materia in raiz.findall('analisis/materias/materia')]
    alertas = [alerta.text for alerta in raiz.findall('analisis/alertas/alerta')]
    
    referencias = [str(referencia.xpath('@referencia')).replace("['",'').replace("']",'') for referencia in raiz.findall('analisis/referencias/anteriores/anterior')]
    referencias_palabras = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/palabra')]
    referencias_texto = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/texto')]

    Item_Id = raiz.xpath(".//identificador/text()")
    Item_name = raiz.xpath('.//titulo/text()') 

    Fecha_Publicacion = datetime.strptime(str(raiz.xpath('.//fecha_publicacion/text()'))[2:-2],"%Y%m%d")
    #Fecha_Publicacion = str(raiz.xpath('.//fecha_publicacion/text()'))[2:-2]
 
    tabla_analisis = tabla_analisis.append({'Item_id': Item_Id,
                                            'Item_Name' : Item_name,
                                            'Fecha_publicacion' : Fecha_Publicacion,
                                            'Materias' : materias,
                                            'Alertas' : alertas,
                                            'Referencias' : referencias,
                                            'Referencias_palabra' : referencias_palabras,
                                            'Referencias_texto' : referencias_texto},
                                            ignore_index=True)


In [ ]:
tabla_analisis.sort_values('Item_id')

In [ ]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
tabla_analisis['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
tabla_analisis['Tags'] = [[] for i in range(len(tabla_analisis))]
tabla_analisis['Match_ASECORP_BBDD'] = [[] for i in range(len(tabla_analisis))]


In [ ]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+','Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+']

In [ ]:
# Consolida las columnas Referencias_palabra y Referencias_texto en una única frase
for i, row in tabla_analisis.iterrows():
    for item_list in range(len(row['Referencias'])): 
        row['Referencias_completas'].append(row['Referencias_palabra'][item_list] + ' ' + row['Referencias_texto'][item_list])
        tabla_analisis['Tags'][i] = re.findall('|'.join(pattern), str(row['Referencias_texto']), flags=re.IGNORECASE)
    print(row['Referencias_completas'])

In [ ]:
tabla_analisis

In [ ]:
texto = ''
for refs in tabla_analisis['Referencias_completas']:
    for ref in refs:
        texto = texto + ref + ' '
        print(ref)

In [ ]:
texto = ''
for i, row in tabla_analisis.iterrows():
    #unique_id = i
    print('\n' + str(row['Item_id']) + str(row['Item_Name']) +'\n')

    antecedente = 1
    for ref in row['Referencias_completas']:
        texto = texto + ref + ' '
        print('\t' + 'Antecedente ' + str(antecedente) + ': ' + ref + ' ' + str(re.findall('|'.join(pattern), ref, flags=re.IGNORECASE)))
        antecedente += 1

In [ ]:
# Aplica expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
regex_result = re.findall('|'.join(pattern), texto, flags=re.IGNORECASE)

In [ ]:
print(regex_result)

In [ ]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [ ]:
boletin_flat_list

## Importa BBDD ASECORP

In [ ]:
ASECORP_BBDD = pd.read_csv('./ASECORP/ExportNormes_20210126.csv', delimiter=';')

In [ ]:
ASECORP_BBDD

In [ ]:
ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'] == 'España']

In [ ]:
ASECORP_BBDD_BOE

In [ ]:
# Crea nueva columna vacía de tipo lista en ASECORP_BBDD
ASECORP_BBDD_BOE['Tags'] = [[] for i in range(len(ASECORP_BBDD_BOE))]

In [ ]:
titulo = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    titulo.append(str(row['Titulo']) + str(re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)))
    ASECORP_BBDD_BOE['Tags'][i] = re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)

In [ ]:
ASECORP_BBDD_BOE[1200:1250]

In [ ]:
# titulo

In [ ]:
len(titulo)

In [ ]:
## Busca patrones regex definidos en columna títulos del DF 
boletin = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    boletin.append(str(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)))

In [ ]:
boletin[0:25]

In [ ]:
## Para cada fila de la BBDD recoge la expresión REGEX encontrada y si no existe no la incluye 
## en la lista resultante llamada 'boletin'. Además si no detecta expresión la cuenta como vacía
## en variable 'n_vacios', y la añade a la lista 'vacios' para inspeccionar posteriormente 
boletin = []
vacios = []
n_vacios = 0
for i, row in ASECORP_BBDD_BOE.iterrows():
    regex_result = re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)
    if len(regex_result) != 0:
        boletin.append(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE))
    else:
        n_vacios += 1
        vacios.append(row['Titulo'])

In [ ]:
n_vacios

In [ ]:
vacios[0:25]

In [ ]:
boletin[0:25]

In [ ]:
# Flatten list of lists
boletin_ASECORP_flat_list = [item for sublist in boletin for item in sublist]

In [ ]:
boletin_ASECORP_flat_list[0:25]

In [ ]:
## Elimina duplicados
boletin_ASECORP_flat_list = list(set(boletin_ASECORP_flat_list))

In [ ]:
boletin_ASECORP_flat_list[0:25]

In [ ]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

In [ ]:
#tabla_analisis['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in tabla_analisis['Tags']:
    for row_comparing in ASECORP_BBDD_BOE['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

In [ ]:
for i, row_to_compare in tabla_analisis.iterrows():
    for j, row_comparing in ASECORP_BBDD_BOE.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            tabla_analisis['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD_BOE['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

In [ ]:
tabla_analisis

In [ ]:
tabla_analisis['Match_ASECORP_BBDD'][1]

In [ ]:
tabla_analisis['Tags'][1]

# Genera Fichero EXCEL de resultados

In [ ]:
## Cambia orden de columnas y elimina las no necesarias  
tabla_analisis_final = tabla_analisis[['Item_id','Item_Name','Fecha_publicacion','Materias','Alertas','Referencias','Referencias_completas','Tags','Match_ASECORP_BBDD']]

In [ ]:
## Crea función que convierte lista a string en todas las columnas de tabla_analisis
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [ ]:
## Aplica función de conversión de listas a strings
tabla_analisis_final = tabla_analisis_final.apply(lambda x: [list2Str(i) for i in x])

In [ ]:
  tabla_analisis_final[0:10]

In [ ]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

tabla_analisis_final_CSV = tabla_analisis_final

In [ ]:
# Compone y genera enlace a PDF del BOE correspondiente
tabla_analisis_final_CSV['Item_id'] = '=HIPERVINCULO(' + '"https://www.boe.es/boe/dias/' \
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%Y')) + '/'\
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%m')) + '/'\
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%d')) + '/'\
                                                        + 'pdfs/'\
                                                        + tabla_analisis_final_CSV['Item_id'] + '.pdf";'\
                                                        + '"' + tabla_analisis_final_CSV['Item_id'] + '")'

In [ ]:
print(tabla_analisis_final_CSV['Item_id'][0])

In [ ]:
# tabla_analisis_final_CSV.to_excel("./ASECORP/Resultados_Matching.xlsx", sheet_name='MATCHING_BOE') 

In [ ]:
tabla_analisis_final_CSV.to_csv("./ASECORP/Resultados_Matching_" + today.strftime("%Y%m%d") + ".csv") 

# ------------------------------------------------------------------------

In [ ]:
print(raiz.tag)

In [ ]:
for materias in raiz.xpath('.//materias'):
    materia = seccion.xpath('.//materia/text()')
    print(materia)

In [ ]:
materias=raiz.findall("analisis/materias/materia")
for materia in materias:
    print(materia.text)

In [ ]:
alertas=raiz.findall("analisis/alertas/alerta")
for alerta in alertas:
    print(alerta.text)

In [ ]:
#tabla_resumen.loc[tabla_resumen['Item_id'] == '[' + BOE[7:23] + ']']
print(tabla_resumen.loc[tabla_resumen['Item_id'] == '[BOE-A-2020-14457]'])

In [ ]:
BOE[7:23]

In [ ]:
tabla_resumen['Item_id']

In [ ]:
tabla_analisis['Item_id']

In [ ]:
tabla_resumen.dtypes

In [ ]:
tabla_resumen.set_index('Item_id').filter(like='14875', axis=0)

In [ ]:
#tabla_resumen = tabla_resumen.astype(str)
#tabla_resumen['Joined'] = tabla_resumen.Item_id.apply(', '.join)
#tabla_resumen['Item_id']=tabla_resumen['Item_id']

In [ ]:
tabla_resumen.dtypes

In [ ]:
tabla_resumen.sort_values('Item_id')

In [ ]:
#for row in tabla_resumen:
#    tabla_resumen.loc[row['Item_id']]['Materias'] = tabla_materias.loc[row['Item_id']]['Materias']

In [ ]:
tabla_resumen['Item_id'] = tabla_resumen['Item_id'].astype(str)
tabla_analisis['Item_id'] = tabla_analisis['Item_id'].astype(str)
tabla_resultados = tabla_resumen.set_index('Item_id').join(tabla_analisis.set_index('Item_id'))


In [ ]:
tabla_resultados

In [ ]:
#tabla_resultados.to_csv('./BOEs/Resultados.csv', index=False)
tabla_resultados.to_csv('./BOEs/Resultados-BOE-' + hoy + '.csv', index=False)

## Genera Split de ALertas y crea una fila para cada una

### Los items sin Alerta se desechan

In [ ]:
# tabla_resultados.Alertas.apply(pd.Series)\
#     .merge(tabla_resultados, left_index = True, right_index = True) \
#         .drop(["Alertas"], axis = 1) \
#             .reset_index() \
#                 .melt(id_vars = ['Item_id','Item_Nombre','Seccion','Departamento','Epigrafe','Item_URL_XML','Materias','Referencias','Referencias_palabra','Referencias_texto'], value_name = "Alerta") \
#                     .drop("variable", axis = 1) \
#                     .dropna() \
#                     .sort_values('Alerta')
                            

In [ ]:
tabla_resultados.Alertas.apply(pd.Series) \
    .merge(tabla_resultados, left_index = True, right_index = True) \
    .drop(["Alertas"], axis = 1) \
    .reset_index() \
    .melt(id_vars = ['Item_id','Item_Nombre','Seccion','Departamento','Epigrafe','Item_URL_XML','Materias','Referencias','Referencias_palabra','Referencias_texto'], value_name = "Alerta") \
    .drop("variable", axis = 1) \
    .dropna() \
    .sort_values('Alerta')

In [ ]:
tabla_resultados['Epigrafe'].str.join(',')

In [ ]:
from pprint import pprint

## Convierte estructura XML en diccionario

In [ ]:
BOE_XML = etree.parse('./BOEs_Anteriores/BOE-A-2021-756.xml')
raiz=BOE_XML.getroot()

In [ ]:
result_list = [item.text for item in raiz.findall("./analisis/referencias/anteriores/anterior/palabra")]
#result_list = [{k: item.get(k) for k in item.iter('materia')} for item in raiz.findall("./analisis/materias")]
#result_list = [{{i.get('k') for i in item.iter('materia')}
#                for k in ('materia')}
#               for item in tree.findall("./analisis/materias")]
#result_list = raiz.findall("./analisis/materias")
pprint(result_list)

In [ ]:
lst = raiz.findall("./analisis/referencias/anteriores/anterior")
d = [{x.tag: {z.tag : z.text for z in x.iter()} for x in c} for c in lst]
pprint(d)

In [ ]:
lst = raiz.findall("./analisis/referencias/anteriores/anterior")
d = [{x.tag: x.text for x in c.iter()} for c in lst]
pprint(d)

In [ ]:
d = [{x.tag: x.text for x in c} for c in raiz]
pprint(d)

In [ ]:
pprint(raiz[2].tag)

In [ ]:
parrafos=raiz.findall("texto/p")
for parrafo in parrafos:
    print(parrafo.text)

# DOGC

In [ ]:
URL_HTML_resumen =  "https://dogc.gencat.cat/es/index.html?newLang=es_ES&language=es_ES"

In [ ]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get(URL_HTML_resumen)

In [ ]:
r.html.links

In [ ]:
r.html.xpath('//*[@id="titol"]/span/text()')

In [ ]:
r.html.xpath('//*[@id="titol"]/h3/text()')


In [ ]:
r.html.lxml.tag

In [ ]:
r.html.lxml.xpath('//*[@id="titol"]/span')

In [ ]:
#import requests
#from lxml import html

response = requests.get(URL_HTML_resumen)

tree = html.fromstring(response.text)

# fetches all useful elements 
useful = tree.xpath('//*[@id="titol"]/span/text()')

print(useful)


In [ ]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

In [ ]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@name')

In [ ]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form/input[1]/@value')

In [ ]:
r.html.lxml.xpath('//*[@id="sumari"]/ul/li[1]/form')

In [ ]:
useful = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/input[3]/@value')

print(useful)

In [ ]:
tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

In [ ]:
cadena = ''
for input in tree.xpath('//*[@id="sumari"]/ul/li[1]/form'):
    argumentos = input.xpath('./input/@name')
    valores = input.xpath('./input/@value')
    print(argumentos, valores)

for indice in range(len(argumentos)):
    cadena += argumentos[indice] + '=' + valores[indice] + '&'

argumentos[1]
cadena[:-1]

## Recoge Nombre Secciones Sumario

In [ ]:
import re
seccion = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/a/text()')
seccion = re.sub('(\\r|\\n|\\t|\ )+', '', seccion[0])
seccion

## Recoge Nombre Secciones Sumario

In [ ]:
secciones = tree.xpath('//*[@id="sumari"]/ul/li')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion = seccion.xpath('./form/a/text()')
    seccion = re.sub('(\\r|\\n|\\t)+', '', seccion[0])
    print(seccion[1:-1])
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion[1:-1]}, ignore_index=True)


In [ ]:
df_secciones_sumarios

## Recoge Valores para formar URLs Secciones Sumario

In [ ]:
URL_base_sumario = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

df_URL_sumarios = pd.DataFrame()
for seccion in secciones:    
    cadena = ''
    for input in seccion.xpath('./form'):
        argumentos = input.xpath('./input/@name')
        valores = input.xpath('./input/@value')
        #print(argumentos, valores)

    for indice in range(len(argumentos)):
        cadena += argumentos[indice] + '=' + valores[indice] + '&'

    URL_sumario = 'https://dogc.gencat.cat' + str(URL_base_sumario[0]) + '?' + str(cadena[:-1])
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

In [ ]:
df_URL_sumarios

In [ ]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [ ]:
df_sumarios

In [ ]:
df_sumarios['URL_Seccion'][4]

## Recoge Items en Seccion Disposiciones

In [ ]:
### Recoge Items en Seccion Disposiciones

response = requests.get(df_sumarios['URL_Seccion'][0])
sumario_HTML = html.fromstring(response.text)

#seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div[1]/text()')
seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div/text()')
bloques = sumario_HTML.xpath('//*[@id="disposicions"]')
df_disposiciones = pd.DataFrame(columns=['Seccion','Item_Name','PDF_link'])
for bloque in bloques: 
    item_name = bloque.xpath('./div/p/text()')
    #item_name = re.sub('(\\r|\\n|\\t)+', '', item_name[0])
    pdf_link = bloque.xpath('./div/div/a[4]/@href')

for row in range(len(pdf_link)):
    df_disposiciones = df_disposiciones.append({'Item_Name': re.sub('(\\r|\\n|\\t)+', '',                                                                               item_name[row]),
                                            'PDF_link' : pdf_link[row],
                                            'Seccion' : seccion[0][:-1]},
                                            ignore_index=True)

df_disposiciones

In [ ]:
pdf_link

# DOUE

In [ ]:
# --------------------------------------------------------------------------------------
# DOUE
# --------------------------------------------------------------------------------------

URL_HTML_resumen =  "https://eur-lex.europa.eu/oj/direct-access.html?locale=es"

## carga página HTML y genera árbol

response = requests.get(URL_HTML_resumen)
tree = html.fromstring(response.text)

## Recoge Nombre Secciones Sumario

secciones = tree.xpath('//*[@id="MainContent"]/div[2]/div[2]/div/table/tbody/tr[1]')
#print(secciones)



In [ ]:
## Recoge Valores para formar URLs Secciones Sumario

df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    seccion_name = seccion.xpath('./td[2]/a/text()')
    seccion_URL = seccion.xpath('./td[2]/a/@href')
    #seccion = re.sub('(\\r|\\n|\\t)+', '', seccion)
    #print(seccion_name)
    #print(seccion_URL)

for row in range(len(seccion_name)):
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion_name[row],
                                            'Seccion_link' : 'https://eur-lex.europa.eu' + seccion_URL[row][4:]},
                                            ignore_index=True)

In [ ]:
df_secciones_sumarios

In [ ]:
[ print (row) for row in df_secciones_sumarios['Seccion_link']]

In [ ]:
df_sumarios = pd.DataFrame()

for URL in df_secciones_sumarios['Seccion_link']:
    
    ### Recoge Items en Seccion Legislación

    #print(URL)
    response = requests.get(URL)
    sumario_HTML = html.fromstring(response.text)

    documentos = sumario_HTML.xpath('//*[@class="oj-ti-doc-dur"]/a[1]/text()[1]')
    documentos_URL = sumario_HTML.xpath('//*[@class="oj-ti-doc-dur"]/a[1]/@href')

    for row in range(len(documentos)):
        df_sumarios = df_sumarios.append({'Seccion': 'L' + documentos_URL[row][-7:-4],
                                        'Item_Name': documentos[row],
                                        'Item_Link': 'https://eur-lex.europa.eu' + documentos_URL[row][10:],
                                        'PDF_Link': ''},
                                        ignore_index=True)


In [ ]:
df_sumarios

In [ ]:
row = 0
for link in df_sumarios['Item_Link']:

    ### Recoge PDF link en Items Legislación

    #print(URL)
    response = requests.get(link)
    sumario_HTML = html.fromstring(response.text)

    PDF_Link = sumario_HTML.xpath('//*[@id="format_language_table_PDF_ES"]/@href')
    df_sumarios['PDF_Link'].iloc[row] = 'https://eur-lex.europa.eu' + PDF_Link[0][10:]

    row += 1
    print(PDF_link)

In [ ]:
df_sumarios = df_sumarios[['Seccion', 'Item_Name', 'Item_Link', 'PDF_Link']]
df_sumarios



## Recoge Valores para formar URLs Secciones Sumario

URL_base_sumario = tree.xpath('//*[@id="sumari"]/ul/li[1]/form/@action')

df_URL_sumarios = pd.DataFrame()
for seccion in secciones:    
    cadena = ''
    for input in seccion.xpath('./form'):
        argumentos = input.xpath('./input/@name')
        valores = input.xpath('./input/@value')
        #print(argumentos, valores)

    for indice in range(len(argumentos)):
        cadena += argumentos[indice] + '=' + valores[indice] + '&'

    URL_sumario = 'https://dogc.gencat.cat' + str(URL_base_sumario[0]) + '?' + str(cadena[:-1])
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

## Concatena en df columnas Sccion y URL de sumarios 

df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

### Recoge Items en Seccion Disposiciones

response = requests.get(df_sumarios['URL_Seccion'][0])
sumario_HTML = html.fromstring(response.text)

#seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div[1]/text()')
seccion = sumario_HTML.xpath('//*[@id="disposicions"]/div/text()')
bloques = sumario_HTML.xpath('//*[@id="disposicions"]')
df_disposiciones = pd.DataFrame(columns=['Seccion','Item_Name','PDF_link'])
for bloque in bloques: 
    item_name = bloque.xpath('./div/p/text()')
    #item_name = re.sub('(\\r|\\n|\\t)+', '', item_name[0])
    pdf_link = bloque.xpath('./div/div/a[4]/@href')

for row in range(len(pdf_link)):
    df_disposiciones = df_disposiciones.append({'Item_Name': re.sub('(\\r|\\n|\\t)+', '', item_name[row]),
                                            'PDF_link' : pdf_link[row],
                                            'Seccion' : seccion[0][:-1]},
                                            ignore_index=True)

#df_disposiciones

def modifica_tabla_resultados_DOGC(tabla):
    tabla = tabla.reset_index(drop=True)
    tabla['Item_Name'] = '<a href=' + tabla['PDF_link'] + ' ' + 'target="_blank"' + '><div>' + tabla['Item_Name'] + '</div></a>' # Añade url link  pdf a Item_id
    tabla.drop(['PDF_link'], axis='columns', inplace=True)
    df_html = tabla.reset_index(drop=True).to_html(index='True', classes="table-hover", escape=False) # Utiliza Clase table-hover de Bootstrap
    df_html = df_html.replace("dataframe", "")  # Elimina clase por defecto dataframe
    df_html = df_html.replace('border="1"', 'border="2"')  # Incrementa tamaño línea borde tabla
    df_html = df_html.replace("<table", '<table style="font-size:12px; text-align: center; width: 100%" ') # Cambia tamaño fuente a 15px
    df_html = df_html.replace("<th>"+ tabla.columns[0], '<th style="text-align: center">'+ tabla.columns[0]) # Cambia alineación a header Columna 1
    df_html = df_html.replace("<th>"+ tabla.columns[1], '<th style="text-align: center">'+ tabla.columns[1]) # Cambia alineación a header Columna 2
    
    return df_html

st.header('')
st.header('DOGC disposiciones')


tabla_resultados_presentacion_DOGC = modifica_tabla_resultados_DOGC(df_disposiciones)

st.write(tabla_resultados_presentacion_DOGC, unsafe_allow_html=True)